In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

url = "http://en.wikipedia.org/wiki/Kevin_Bacon"

html = urlopen(url)
bsObj = BeautifulSoup(html,"html.parser")

# 指向其它词条的链接的共同点。
# 1： 都是在id为 bodyContent的div标签里。
# 2： url链接不包含冒号。
# 3： url 均以/wiki/开头。


for link in bsObj.find("div",{"id":"bodyContent"}).findAll("a", href=re.compile("^(/wiki/)((?!:).)*$")):
    if "href" in link.attrs:
        print(link.attrs["href"])


/wiki/Kevin_Bacon_(disambiguation)
/wiki/San_Diego_Comic-Con
/wiki/Philadelphia
/wiki/Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/HBO
/wiki/Taking_Chance
/wiki/Golden_Globe_Award
/wiki/Screen_Actors_Guild_Award
/wiki/Primetime_Emmy_Award
/wiki/The_Guardian
/wiki/Academy_Award
/wiki/Hollywood_Walk_of_Fame
/wiki/Social_networks
/wiki/Six_Degrees_of_Kevin_Bacon
/wiki/SixDegrees.org
/wiki/Philadelphia
/wiki/Edmund_Bacon_(architect)
/wiki/Pennsylvania_Governor%27s_School_for_the_Arts
/wiki/Bucknell_University
/wiki/Glory_Van_Scott
/wiki/Kevin_Bacon_filmography
/wiki/Circle_in_the_Square
/wiki/Nancy_Mills
/wiki/Cosmopolitan_(magazine)
/wiki/Fraternities_and_sororities
/wiki/Animal_House
/wiki/Search_for_Tomorr

In [10]:
# getLinks 函数可以用维基百科词条 wiki/<词条名称>形式的url链接作为参数。以同意的形式返回一个列表。里面包含所有的词条url链接。

# 一个主函数。以某个初始词条为参数调用getLinks。再从返回的URL列表里。随机选择一个词条链接。再调用getLinks。直到我们主动停止。或者在新的页面上没有找到词条链接。才停止。

from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())

def getLinks(articleUrl):
    linksList = []
    html = urlopen("http://en.wikipedia.org" + articleUrl)
    bsObj = BeautifulSoup(html, "html5lib")
    links = bsObj.find("div",{"id":"bodyContent"}).findAll("a",href = re.compile("^(/wiki/)((?!:).)*$"))
    for link in links:
        if "href" in link.attrs:
            linksList.append(link.attrs["href"])
    return linksList

links = getLinks("/wiki/kevin_Bacon")
print(links)
# 不断爬取，直到列表为空。
# 要想自动化爬虫，需要加入更为完善的异常处理机制。

while len(links) > 0:
    newAtticle = links[random.randint(0, len(links)-1)]
    # 另一种写法。get返回的是一整个元素，在这句解析出 “wiki/*” 耗时似乎会更少。
    # newAtticle = links[random.randint(0, len(links)-1)].attrs["href"]
    print(newAtticle)
    links = getLinks(newAtticle)

['/wiki/Kevin_Bacon_(disambiguation)', '/wiki/San_Diego_Comic-Con', '/wiki/Philadelphia', '/wiki/Pennsylvania', '/wiki/Kyra_Sedgwick', '/wiki/Sosie_Bacon', '/wiki/Edmund_Bacon_(architect)', '/wiki/Michael_Bacon_(musician)', '/wiki/Footloose_(1984_film)', '/wiki/JFK_(film)', '/wiki/A_Few_Good_Men', '/wiki/Apollo_13_(film)', '/wiki/Mystic_River_(film)', '/wiki/Sleepers', '/wiki/The_Woodsman_(2004_film)', '/wiki/Fox_Broadcasting_Company', '/wiki/The_Following', '/wiki/HBO', '/wiki/Taking_Chance', '/wiki/Golden_Globe_Award', '/wiki/Screen_Actors_Guild_Award', '/wiki/Primetime_Emmy_Award', '/wiki/The_Guardian', '/wiki/Academy_Award', '/wiki/Hollywood_Walk_of_Fame', '/wiki/Social_networks', '/wiki/Six_Degrees_of_Kevin_Bacon', '/wiki/SixDegrees.org', '/wiki/Philadelphia', '/wiki/Edmund_Bacon_(architect)', '/wiki/Pennsylvania_Governor%27s_School_for_the_Arts', '/wiki/Bucknell_University', '/wiki/Glory_Van_Scott', '/wiki/Kevin_Bacon_filmography', '/wiki/Circle_in_the_Square', '/wiki/Nancy_Mills

/wiki/Ross_Katz
/wiki/John_Korty
/wiki/Alex_Segal
/wiki/Corey_Allen
/wiki/Supercarrier_(TV_series)
/wiki/Alex_Hyde-White
/wiki/The_Merv_Griffin_Show
/wiki/Tom_Cruise
/wiki/Purification_Rundown
/wiki/International_Standard_Book_Number


KeyboardInterrupt: 

In [18]:
# 采集整个网站的好处。
# 生成网站地图，找出网站不曾被人留意的部分，加以改善。
# 收集数据。收集有价值网站上的新闻数据。

# 网站采集办法，一般是从顶级页面开始（比如主页），然后搜索页面上的所以链接，形成列表。再去采集这些链接的每一个页面。
# 然后再把每个页面上找到的裂解形成新的列表。。重复执行下一轮采集。

# 为了避免一个页面被采集了两次，链接重是非常必要的。在代码运行时，把已发现的所有链接放在一起，并保存在方便查询的列表里。（set集合）
# 只有“新”的链接才会被采集。之后再从页面中采集新的链接。

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set()

def getLinks(pageUrl):
    global pages
    html = urlopen("http://en.wikipedia.org" + pageUrl)
    bsObj = BeautifulSoup(html, "html5lib")
    for link in bsObj.findAll("a", href=re.compile("^(/wiki/)")):
        if "href" in link.attrs:
            if link.attrs["href"] not in pages:
                # 如果链接不在 pages集合中，表示遇到了新页面。
                newPage = link.attrs["href"]
                print("我遇到了新链接，并进行下一步提取/n{0}".format(newPage))
                pages.add(newPage)
                getLinks(newPage)
getLinks("")

# 会查找所有以 /wiki/ 开头的链接。不考虑链接是否包含冒号：

# 词条链接不包含冒号，文档上传页，讨论之类的页面的URL链接都包含冒号。
 
# 开始时，用getLinks处理一个空链接。其实就是维基的首页。（http:wikipedia.org + ""）
# 遍历首页上的每个链接。并检查url 是否已经在集合page中了。如果不在就答应在屏幕上。并把链接加到集合中。递归处理这个链接。

# python 默认的递归限制（程序的自我调用次数）是1000次。如果递归的运行次数非常多，前面的递归程序可能会崩溃。
# 维基的网络链接瀚如烟海，所以这个程序到了递归限制会自动停下，除非是设置一个较大的递归计数器，或用其它手段不让他停止。

我遇到了新链接，并进行下一步提取/n/wiki/Wikipedia
我遇到了新链接，并进行下一步提取/n/wiki/Wikipedia:Protection_policy#semi
我遇到了新链接，并进行下一步提取/n/wiki/Wikipedia:Requests_for_page_protection
我遇到了新链接，并进行下一步提取/n/wiki/Wikipedia:Requests_for_permissions
我遇到了新链接，并进行下一步提取/n/wiki/Wikipedia:Requesting_copyright_permission


KeyboardInterrupt: 